In [1]:
#!pip install chart_studio
import common
import pandas as pd
import haversine as hs


In [2]:
df = common.df_from_list()
df.to_hdf('df.h5', 'df')
df

,datetime,duration,sla,slo,ela,elo
0,2023-01-03 12:11:00,9.650000,49.197684,16.640210,49.189396,16.617106
1,2023-01-03 17:57:55,11.550000,49.189396,16.617106,49.201914,16.641787
2,2023-01-04 11:48:46,10.550000,49.201914,16.641787,49.191415,16.619807
3,2023-01-04 17:53:59,12.533333,49.189396,16.617106,49.201914,16.641787
4,2023-01-09 11:49:58,11.033333,49.201914,16.641787,49.191415,16.619807
...,...,...,...,...,...,...
406,2024-05-07 12:36:15,2.550000,49.189396,16.617106,49.191415,16.619807
407,2024-05-07 13:27:16,7.266667,49.191415,16.619807,49.196597,16.608866
408,2024-05-07 18:16:10,13.450000,49.196597,16.608866,49.195861,16.650252
409,2024-05-08 10:14:22,11.533333,49.195861,16.650252,49.200186,16.631748


In [11]:
# we use haversine package to calculate the distance between two points
df['distance'] = df.apply(lambda x: hs.haversine((x['sla'], x['slo']), (x['ela'], x['elo']), unit=hs.Unit.KILOMETERS), axis=1)

# I measured that travel distance is 1.36 times the aerial distance
df['map_distance'] = df.distance * 1.36

df = df[df.distance < 100]
df = df[df.distance > 0]

In [12]:
month_agg = common.time_agg(df, 'M', "%Y-%m")
month_agg.index.rename('date', inplace=True)
pd.set_option('display.precision', 2)
month_agg

,distance,duration,trips
date,,,
2023-01,85.46,353.90,33
2023-02,60.85,288.62,32
2023-03,75.12,317.75,44
2023-04,57.47,230.88,25
2023-05,72.79,614.47,30
2023-06,13.73,52.67,6
2023-09,54.79,219.03,17
2023-10,105.67,404.45,33
2023-11,129.98,520.22,37


In [17]:
day_agg = common.time_agg(df, 'D', "%Y-%m-%d")
day_agg.index.rename('date', inplace=True)

print("Average distance per day: %.2f" % day_agg.distance.mean())
print("Average number of trips per day: %.2f" % day_agg.trips.mean())
print("Average time spent on bike per day: %.2f" % day_agg.duration.mean(), "min")
day_agg


Average distance per day: 4.94
Average number of trips per day: 1.78
Average time spent on bike per day: 21.48 min


,distance,duration,trips
date,,,
2023-01-03,5.69,21.20,2
2023-01-04,5.78,23.08,2
2023-01-09,5.78,24.05,2
2023-01-10,4.36,18.05,2
2023-01-11,6.40,26.72,3
...,...,...,...
2024-04-29,7.48,29.33,2
2024-04-30,3.17,11.28,1
2024-05-07,9.64,40.28,4
